In [13]:
import sys
from core import *
from parser import *

In [176]:
class Mapper:
    
    def __init__(self, context, diagram):
        self.context = context
        self.diagram = diagram
        self.mapping = {}
        self.options = [{}]
        
    def combine_mappings(self, A, B):
        C = A.copy()
        for x in B:
            if x in A and A[x] != B[x]:
                return None
            C[x] = B[x]
        return C
    
    def combine_lists_of_options(self, options_A, options_B):
        options = []
        for B in options_B:
            for A in options_A:
                C = self.combine_mappings(A, B)
                if C != None:
                    options.append(C)
        
        # remove duplicates ?
        return options
        
    # Find all possible ways in which x could equal y
    def find_consequences(self, x, y):
        if x in self.context.data:
            return [{}]
        
        reps_x = [ r for r in self.context.representations if r.ptr == x ]
        reps_y = [ r for r in self.diagram.representations if r.ptr == y ]
        
        options = []
        for r_x in reps_x:
            for r_y in reps_y:
                m = self.reps_match_consequences(r_x, r_y)
                if m != None:
                    options.append(m)
                    
        # remove duplicate options ?
        return options
    
    # Returns what the consequence is if r_x matches r_y
    def reps_match_consequences(self, r_x, r_y):
        if type(r_x) != type(r_y):
            return None
        
        if isinstance(r_x, Repr_Property):
            if r_x.prop != r_y.prop:
                return False

        m = {}
        r_x_deps = r_x.dependencies()
        r_y_deps = r_y.dependencies()
        
        for u, v in zip(r_x_deps, r_y_deps):
            if self.context.owns(u):
                m[u] = v
                
        return m
    
    def mapping_set_multiple(self, M):
        for x in M:
            if not self.mapping_set(x, M[x]):
                return False
        return True
    
    def mapping_set(self, x, y):
#         print('Want to map {} --> {}'.format(self.context.str_x(x), self.diagram.str_x(y)))
        
        # If x was already mapped, make sure it was mapped to y
        if x in self.mapping:
            return self.mapping[x] == y
        
        self.mapping[x] = y
        
        # Induced mapping for category, domain, codomain (that is, if they need to be mapped)
        if self.context.owns(x.category) and not self.mapping_set(x.category, y.category):
            return False
        
        if not x.is_object():
            if self.context.owns(x.domain) and not self.mapping_set(x.domain, y.domain):
                return False

            if self.context.owns(x.codomain) and not self.mapping_set(x.codomain, y.codomain):
                return False
        
        # See what the consequences are from mapping x to y
        x_options = self.find_consequences(x, y)
        
        # If there is just one x_option, immediately update mapping
        if len(x_options) == 1:            
            if not self.mapping_set_multiple(x_options[0]):
                return False
        
        # Combine old options with the new options
        self.options = self.combine_lists_of_options(self.options, x_options)
                        
        # If there are no options, return False
        if not self.options:
            return False
        
        # Again, if there is just one option, immediately update mapping
        if len(self.options) == 1:
            if not self.mapping_set_multiple(self.options[0]):
                return False
                
            # Also reset options
            self.options = [{}]
            
        return True

In [177]:
G.theorems

{'affine_implies_qcompact': <core.Theorem at 0x111204d90>}

In [178]:
# diagram = Diagram()
# diagram.add_reference(G)

In [182]:
thm = G.theorems['affine_implies_qcompact']
M = Mapper(thm, diagram)
M.mapping_set(aff_thm_X, aff_X)

In [183]:
M.mapping

{}

In [184]:
for x in M.mapping:
    print('{} --> {}'.format(thm.str_x(x), diagram.str_x(M.mapping[x])))

In [105]:
M.mapping

{}

In [33]:
G = recreate_global_diagram()

stream = Stream(open('/Users/jessetvogel/Projects/abstract-nonsense/math/example_theorems.txt'))
lexer = Lexer(stream)
parser = Parser(lexer)
parser.book = G

In [34]:
parser.parse()